This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google earth engine account

Follow this [link](https://code.earthengine.google.com/register) to create a GEE account

## Step 2: Installation


In [1]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git@crs_transform

  Cloning https://github.com/Mining-for-the-Future/ASTER_preprocessing.git (to revision crs_transform) to c:\users\eweav\appdata\local\temp\pip-req-build-dirzx_pi
  Resolved https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to commit bcf0e7c53a839093bd90bc2aec7bd8beeaa1e71e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Mining-for-the-Future/ASTER_preprocessing.git 'C:\Users\eweav\AppData\Local\Temp\pip-req-build-dirzx_pi'
  Running command git checkout -b crs_transform --track origin/crs_transform
  branch 'crs_transform' set up to track 'origin/crs_transform'.
  Switched to a new branch 'crs_transform'


## Step 3: Importing Libraries

In [ ]:
%pip install geemap

importing these modules should trigger the google earth engine authentication process.

Note: in VS Code, the input field for the google earth engine token is at the top of the window, not underneath the cell.

In [2]:
from ASTER_preprocessing.preprocessing import aster_collection_preprocessing, aster_image_preprocessing
from ASTER_preprocessing.vis_params import vis_params_image


Successfully saved authorization token.


In [3]:
import geemap

## Step 4: Preprocess ASTER image collection using a drawn polygon

In [4]:
# Create an interactive map
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
# Get the last drawn feature
geometry = Map.user_roi

In [7]:
collection = aster_collection_preprocessing(geometry, bands = ['B04', 'B3N', 'B02'], masks = [])

No TIR bands specified
No TIR bands specified
No TIR bands specified


In [8]:
collection.size().getInfo()

47

## Step 4A: preprocess aster image from drawn point

In [10]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
geometry = Map.user_roi

### Examine the dates of images intersecting this point to select one

In [12]:
from ASTER_preprocessing.preprocessing import ee_i

In [ ]:
from datetime import datetime

In [14]:
collection = ee_i.ImageCollection('ASTER/AST_L1T_003').filterBounds(geometry)

In [ ]:
image_list = collection.toList(collection.size())

In [ ]:
dates = []
for i in range(image_list.length().getInfo()):
    image = ee_i.Image(image_list.get(i))
    date = datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime('%Y%j.%H%M')
    dates.append(date)

In [ ]:
'%Y%j.%H%M'

In [35]:
dates

['2000120.1732',
 '2000120.1732',
 '2000152.1731',
 '2000152.1731',
 '2000200.1731',
 '2000200.1731',
 '2000232.1731',
 '2000232.1731',
 '2000257.1725',
 '2000328.1729',
 '2000328.1729',
 '2000344.1728',
 '2000344.1729',
 '2001010.1728',
 '2001010.1728',
 '2001026.1728',
 '2001042.1728',
 '2001042.1728',
 '2001138.1726',
 '2001138.1726',
 '2001250.1723',
 '2001250.1723',
 '2001323.1714',
 '2002269.1717',
 '2003224.1715',
 '2003224.1715',
 '2005005.1715',
 '2005005.1715',
 '2005037.1715',
 '2005037.1715',
 '2005069.1715',
 '2005069.1715',
 '2005085.1715',
 '2005085.1716',
 '2006152.1715',
 '2006152.1715',
 '2006216.1716',
 '2006216.1716',
 '2006328.1715',
 '2006328.1716',
 '2007027.1716',
 '2007027.1716',
 '2007091.1716',
 '2007091.1716',
 '2007139.1716',
 '2007139.1716',
 '2007149.0417',
 '2007155.1716',
 '2007155.1716',
 '2007171.1716',
 '2007171.1716',
 '2007247.0405',
 '2007324.1710',
 '2008010.0405',
 '2008190.1716',
 '2008190.1716',
 '2008222.1716',
 '2008222.1716',
 '2008231.1710

In [15]:
collection.filterDate('2005-03-26', '2005-03-27').size().getInfo()

1

In [16]:
image = collection.filterDate('2005-03-26', '2005-03-27').first()

In [17]:
def draw_map(img):
    map = geemap.Map()
    map.centerObject(img)
    map.addLayer(img)
    return map

In [18]:
draw_map(image)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [25]:
image.bandNames().getInfo()

['B01',
 'B02',
 'B3N',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14']

In [19]:
image.get('CLOUDCOVER').getInfo()

0

In [27]:
from ASTER_preprocessing.data_conversion import aster_dn2toa

In [28]:
image_toa = aster_dn2toa(image, ['B3N', 'B04', 'B05'])
map = geemap.Map()
map.center_object(image_toa)
map.addLayer(image_toa, vis_params = {'bands': ['B3N', 'B04', 'B05'], 'min': [0, 0, 0], 'max': [1, 1, 1]})
map

No TIR bands specified


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
image_proc = aster_image_preprocessing(image, ['B3N', 'B02', 'B01'], masks = ['cloud'])
vis_params = vis_params_image(image_proc, ['B3N', 'B02', 'B01'])

map = geemap.Map()
map.center_object(image_proc)
map.addLayer(image_proc, vis_params = vis_params, name = 'processed_image')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
image = aster_image_preprocessing(image, bands = ['B3N', 'B04', 'B05'], masks = [])

No TIR bands specified


In [ ]:
image.bandTypes().getInfo()

In [31]:
vis_params_image(image, ['B3N', 'B04', 'B05'])

{'min': [0.11821396399470005, 0.08883829686909374, 0.06885419034135193],
 'max': [0.29887389363836125, 0.24521091197755646, 0.17596070865010924],
 'bands': ['B3N', 'B04', 'B05']}

In [ ]:
def vis_params_image(image, bands):
    geometry = image.geometry()
    projection = image.select(bands[0]).projection().getInfo()
    
    percentiles = image.select(bands).reduceRegion(
        reducer = ee_i.Reducer.percentile([5, 95]),
        geometry = geometry,
        crs = projection['crs'],
        crsTransform = projection['transform'],
        bestEffort = True).getInfo()

    b_p5 = [band + '_p5' for band in bands]
    b_p95 = [band + '_p95' for band in bands]
    mins = [percentiles[min] for min in b_p5]
    maxs = [percentiles[max] for max in b_p95]
    vis_params = {'min': mins, 'max': maxs, 'bands': bands}
    return vis_params

In [ ]:
image.geometry().getInfo()

In [32]:
map = geemap.Map()
map.centerObject(image)
map.addLayer(image)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
image.select('B01').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32615',
 'transform': [15, 0, 453413, 0, -15, 4984928]}

## Step 5: Visualize the image collection as a time series

In [ ]:
first_image = collection.first()
vis_params = vis_params_image(first_image, bands = ['B04', 'B3N', 'B02'])

map2 = geemap.Map()
map2.centerObject(first_image)
# map2.addLayer(first_image, vis_params)
map2.add_time_slider(collection, vis_params = vis_params, time_interval = 1)
map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

No such comm: 018dc01c69cc4f8d9487bf419b5bd225
No such comm: 018dc01c69cc4f8d9487bf419b5bd225
